In [28]:
.libPaths("~/R/x86_64-pc-linux-gnu-library/4.0/")

In [80]:
setwd("~/LAJC-expungement/")
here::here()

[1] "/home/jupyter-seth127/LAJC-expungement"

In [134]:
install.packages("RPostgres")

Installing package into ‘/home/jupyter-seth127/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

also installing the dependencies ‘blob’, ‘lubridate’, ‘Rcpp’




In [46]:
library(DBI)
library(dplyr)
library(here)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [27]:
postPass=Sys.getenv("POSTGRES_PASS")
con <- dbConnect(
  RPostgres::Postgres(),
  dbname = 'expunge',
  host = 'localhost',
  port = '5432',
  password = postPass,
  user = 'jupyter'
)

In [135]:
res <- dbSendQuery(con, "SELECT COUNT(person_id) FROM expunge")
print(dbFetch(res))
dbClearResult(res)

    count
1 9054266


In [30]:
res <- dbSendQuery(con, "SELECT DISTINCT person_id FROM expunge")
all_ids <- dbFetch(res)
dbClearResult(res)
head(all_ids)

,person_id
,<chr>
1,140001000000344
2,167181000000324
3,135180000000868
4,141180000000479
5,29181000000654
6,176010000001172


In [31]:
nrow(all_ids)

[1] 3082954

In [48]:
set.seed(12345)
ids_1k <- sample_n(all_ids, 1000)
ids_10k <- sample_n(all_ids, 10000)
ids_100k <- sample_n(all_ids, 100000)

In [52]:
dbWriteTable(con, "ids_1k_sample", ids_1k)
dbWriteTable(con, "ids_10k_sample", ids_10k)
dbWriteTable(con, "ids_100k_sample", ids_100k)

Warning message in result_create(conn@ptr, statement, immediate):
“Closing open result set, cancelling previous query”


In [61]:
dbExecute(con, "
CREATE TABLE data_1k_sample AS
SELECT * FROM ids_1k_sample
JOIN expunge USING (person_id)
")

[1] 293598

In [61]:
dbExecute(con, "
CREATE TABLE data_10k_sample AS
SELECT * FROM ids_10k_sample
JOIN expunge USING (person_id)
")

[1] 293598

In [61]:
dbExecute(con, "
CREATE TABLE data_100k_sample AS
SELECT * FROM ids_100k_sample
JOIN expunge USING (person_id)
")

[1] 293598

In [121]:
res <- dbSendQuery(con, "SELECT * FROM data_1k_sample")
df_1k <- dbFetch(res)
dbClearResult(res)
print(nrow(df_1k))
head(df_1k$person_id)

[1] 2811


[1] "199031000000439" "15100000000316"  "15100000000316"  "10210000000095" 
[5] "51220000000305"  "51220000000305"

In [114]:
read_person_file_db <- function(.id, .table, .con) {
    res <- dbSendQuery(.con, glue::glue('
      SELECT 
      "person_id",
      "HearingDate",
      "CodeSection",
      "ChargeType",
      "Class",
      "DispositionCode",
      "Plea",
      "Race",
      "Sex",
      "fips"
      FROM {.table} a WHERE person_id = \'{.id}\''))
    df <- dbFetch(res)
    dbClearResult(res)
    return(df)
}

In [128]:
id <- "51220000000305"
data <- read_person_file_db(id, "data_1k_sample", con)
data

person_id,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
51220000000305,2010-05-03,46.2-613(2),Misdemeanor,1,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840
51220000000305,2010-05-03,A.46.2-707,Misdemeanor,3,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840
51220000000305,2010-05-03,46.2-300,Misdemeanor,2,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840


In [129]:
source(here("code", "expunge_classifier2.R"))

In [131]:
read_person_file_db(id, "data_1k_sample", con) %>%
  classify_ex2()

Warning message:
“Unknown levels in `f`: Nolle Prosequi, No Indictment Presented, Not True Bill, Dismissed/Other, Not Guilty, Not Guilty/Acquitted, Guilty In Absentia”


person_id,HearingDate,CodeSection,codesection,ChargeType,chargetype,Class,DispositionCode,disposition,Plea,⋯,within7,within10,class1_2,class3_4,expungable,old_expungable,expungable_no_lifetimelimit,reason,sameday,lifetime
<chr>,<date>,<chr>,<fct>,<chr>,<fct>,<chr>,<chr>,<fct>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<fct>,<lgl>,<fct>,<chr>,<lgl>,<lgl>
51220000000305,2010-05-03,46.2-613(2),covered elsewhere,Misdemeanor,Misdemeanor,1,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,Automatic,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date,FALSE,FALSE
51220000000305,2010-05-03,A.46.2-707,covered elsewhere,Misdemeanor,Misdemeanor,3,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,Automatic,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date,FALSE,FALSE
51220000000305,2010-05-03,46.2-300,covered elsewhere,Misdemeanor,Misdemeanor,2,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Not eligible,FALSE,Automatic,"Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date; HOWEVER, the outcome is changed to not eligible because the lifetime limit of two expungements has been exceeded",FALSE,TRUE


In [122]:
#setwd("~/Box Sync/Code for Cville")

library(randomForest)
#library(tidyverse)
library(dplyr)
library(readr)
library(tidyr)
library(here)
library(forcats)
#source(here("code", "helper-functions.R"))
NODE_ENCODE <- readr::read_csv(here("data", "reasons_encode.csv"), col_types = "ic")

########## Load data and trained classifier ##########

#classify_ex <- function(id){

  load(here("data", "expunge_coder.Rdata"))



In [123]:
id <- "51220000000305"
  
  #data <- read_person_file(id, '') %>%
data <- read_person_file_db(id, "data_1k_sample", con) %>%
    replace_na(list(CodeSection = "MISSING"))

  orig_cols <- colnames(data)


  
  ########## Data cleaning ##########
  
  data <- data %>% 
    filter(!is.na(DispositionCode)) %>%
    filter(ChargeType %in% c("Misdemeanor", "Felony")) %>%
    filter(
      DispositionCode %in% c(
        "Guilty",
        "Guilty In Absentia",
        "Dismissed",
        "Nolle Prosequi",
        "Not Guilty",
        "Not Guilty/Acquitted",
        "No Indictment Presented",
        "Not True Bill",
        # "Resolved", #??
        "Dismissed/Other"
      )
    )

    if(nrow(data) == 0) return(data)
  
  ########## Generate objects to assist in feature construction ########
  
  A <- c("4.1-305", "18.2-250.1")
  B <- c("4.1-305","18.2-96","18.2-103","18.2-119","18.2-120","18.2-134","18.2-250.1","18.2-415")
  Bmis <- c("18.2-248.1")
  Twelve <- c("18.2-36.1","18.2-36.2","18.2-51.4","18.2-51.5","18.2-57.2","18.2-266","46.2-341.24")
  
  #Generate time-distance matrices
  dates <- as.numeric(as.Date(data$HearingDate))
  d <- outer(dates, dates, "-") # difference in days between all dates
  
  d_fel <- d # isolate dates within the last ten years
  d_fel[d > 365.25*10 | d < 0] <- 0
  d_fel[d_fel != 0] <- 1
  
  d_arr <- d # isolate dates within the last three years
  d_arr[d > 365.25*3 | d < 0] <- 0
  d_arr[d_arr != 0] <- 1
  
  d7 <- d # isolate dates within the next seven years
  d7[d > 0 | d < -365.25*7] <- 0
  d7[d7 != 0] <- 1
  
  d10 <- d # isolate dates within the next ten years
  d10[d > 0 | d < -365.25*10] <- 0
  d10[d10 != 0] <- 1
  
  ########## FEATURE CONSTRUCTION ##########
  
  # Disposition = {Convicted; Deferred Dismissal; Dismissed}
  # Deferred dismissal if disposition = Dismissed and plea = Guilty
  # Plea = {guilty; not guilty}
  # Must combine nolle prosequi and other pleas into these two categories
  
  data <- data %>%
    mutate(disposition = DispositionCode,
           disposition = fct_recode(disposition,
                                    "Dismissed" = "Nolle Prosequi",
                                    "Dismissed" = "No Indictment Presented",
                                    "Dismissed" = "Not True Bill",
                                    "Dismissed" = "Dismissed/Other",
                                    "Dismissed" = "Not Guilty",
                                    "Dismissed" = "Not Guilty/Acquitted",
                                    "Conviction" = "Guilty In Absentia",
                                    "Conviction" = "Guilty"),
           disposition = fct_expand(disposition, "Conviction", "Dismissed", "Deferral Dismissal")
    )
  dd <- data$Plea %in% c("Alford", "Guilty", "Nolo Contendere") & data$disposition == "Dismissed"
  data$disposition[dd] <- "Deferral Dismissal"
  
  # Chargetype = {Felony; Misdemeanor}
  
  data <- data %>%
    mutate(chargetype = factor(ChargeType, levels = c("Misdemeanor", "Felony")),
           chargetype = fct_expand(chargetype, "Misdemeanor", "Felony"))
  
  # Codesection = {covered in 19.2-392.6 A; covered in 19.2-392.6 B; excluded by 19.2-392.12; covered elsewhere}
  
  data <- data %>%
    mutate(codesection = "covered elsewhere",
           codesection = ifelse(CodeSection %in% B | (CodeSection %in% Bmis & chargetype == "Misdemeanor"), "covered in 19.2-392.6 - B", codesection),
           codesection = ifelse(CodeSection %in% A & disposition == "Deferral Dismissal", "covered in 19.2-392.6 - A", codesection),
           codesection = ifelse(CodeSection %in% Twelve, "excluded by 19.2-392.12", codesection),
           codesection = as.factor(codesection),
           codesection = fct_expand(codesection, "covered in 19.2-392.6 - A",
                                    "covered in 19.2-392.6 - B",
                                    "excluded by 19.2-392.12",
                                    "covered elsewhere"))
  
  # Convictions = {True; False} (convictions on the person's record)
  
  data <- data %>%
    mutate(convictions = any(disposition == "Conviction"))
  
  # Arrests = {True; False} (arrests or charges in the past 3 years)
  
  data <- data %>%
    mutate(arrests = (rowSums(d_arr)!=0))
  
  # Felony10 = {True;False} (felony convictions within the last 10 years)
  
  data <- data %>%
    mutate(felony10 = as.logical(d_fel %*% (chargetype == 'Felony' & disposition == "Conviction")) > 0)
  
  # Sevenyear = {True; False} (convictions of another kind within 7 years from disposition date)
  # Tenyear = {True; False} (convictions of another kind within 10 years from disposition date)
  
  data <- data %>%
    mutate(sevenyear = as.logical(d7 %*% as.matrix(disposition == "Conviction")) > 0,
           tenyear = as.logical(d10 %*% as.matrix(disposition == "Conviction")) > 0)
  
  # Within7 = {True; False} (disposition date is within 7 years of the current date)
  # Within10 = {True; False} (disposition date is within 10 years of the current date)
  
  data <- data %>%
    mutate(HearingDate = as.Date(HearingDate),
           within7 = (HearingDate > (as.Date("2020-12-31") - lubridate::years(7))),
           within10 = (HearingDate > (as.Date("2020-12-31") - lubridate::years(10))))
  
  # Class3_4 = {True; False} (class 3 or 4 felony conviction within the past 20 years)
  # Class1_2 = {True; False} (class 1 or 2 felony or any other felony punishable by imprisonment for life)
  
  data <- data %>%
    mutate(class1_2 = any(Class %in% c("1", "2") & chargetype=="Felony"),
           class1_2 = ifelse(is.na(class1_2), FALSE, class1_2),
           class3_4 = any(Class %in% c("3", "4") & chargetype=="Felony"),
           class3_4 = ifelse(is.na(class3_4), FALSE, class3_4))

data

Warning message in result_create(conn@ptr, statement, immediate):
“Closing open result set, cancelling previous query”
Warning message:
“Unknown levels in `f`: Nolle Prosequi, No Indictment Presented, Not True Bill, Dismissed/Other, Not Guilty, Not Guilty/Acquitted, Guilty In Absentia”


person_id,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,⋯,codesection,convictions,arrests,felony10,sevenyear,tenyear,within7,within10,class1_2,class3_4
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<fct>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
51220000000305,2010-05-03,46.2-613(2),Misdemeanor,1,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,covered elsewhere,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
51220000000305,2010-05-03,A.46.2-707,Misdemeanor,3,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,covered elsewhere,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
51220000000305,2010-05-03,46.2-300,Misdemeanor,2,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,covered elsewhere,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [124]:

  ############## Apply the Random Forest to perform classification here ############
  
  # apply the auto encoder:
  data <- data %>%
    mutate(expungable = predict(expunge_coder, newdata=data))
  
  # add old law expungable count
  data <- data %>% 
    mutate(old_expungable = (disposition == "Dismissed"))

  # rationales
  tree <- getTree(expunge_coder, labelVar=TRUE) #tree
  nodes <- attr(predict(expunge_coder, newdata=data, nodes=TRUE), "nodes")
  data$node <- nodes
  data <- left_join(data, NODE_ENCODE, by="node")
  data

person_id,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,⋯,sevenyear,tenyear,within7,within10,class1_2,class3_4,expungable,old_expungable,node,reason
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<fct>,<lgl>,"<int[,1]>",<chr>
51220000000305,2010-05-03,46.2-613(2),Misdemeanor,1,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,32,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date
51220000000305,2010-05-03,A.46.2-707,Misdemeanor,3,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,32,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date
51220000000305,2010-05-03,46.2-300,Misdemeanor,2,Guilty,Guilty,White Caucasian(Non-Hispanic),Male,840,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,32,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date


In [127]:

  ############# Additional conditions that change the expungement outcome ##########
  
  # Sameday = {True; False} (same day rule in effect)
  
  data <- data %>%
    group_by(HearingDate) %>%
    mutate(nonauto_count_day = sum(expungable != "Automatic" & expungable != "Automatic (pending)")) %>%
    ungroup() %>%
    mutate(sameday = (expungable %in% c("Automatic", "Automatic (pending)") & nonauto_count_day > 0),
           reason2 = ifelse(sameday, "; HOWEVER, the outcome is changed to petition because of a conviction on the same day of something that is not automatically expungable",""))
  
  data$expungable[data$sameday & data$expungable=="Automatic"] <- "Petition"
  data$expungable[data$sameday & data$expungable=="Automatic (pending)"] <- "Petition (pending)"
  data$expungable_no_lifetimelimit <- data$expungable
  
  # Lifetime = {True; False} (more than 2 lifetime expungements of convictions)
  
  data <- data %>%
    arrange(HearingDate) %>%
    mutate(totalexpunge = cumsum(expungable %in% c("Automatic", "Petition")),
           lifetime = totalexpunge > 2,
           lifetime = lifetime & (disposition == "Conviction" | (disposition == "Deferral Dismissal" & sameday)),
           reason2 = ifelse(lifetime & expungable != "Not eligible", "; HOWEVER, the outcome is changed to not eligible because the lifetime limit of two expungements has been exceeded","")) %>%
    unite(reason, reason, reason2, sep="")
  
  data$expungable[data$lifetime] <- "Not eligible"
  
  # Fix missing values for race and sex: replace with mode
  if (all(is.na(data$Race))) {
    data <- mutate(data, Race = "MISSING")
  } else {
    data$Race[is.na(data$Race)] <- names(which.max(table(data$Race)))[1]  
  }
  
  if (all(is.na(data$Sex))) {
    data <- mutate(data, Sex = "MISSING")
  } else {
    data$Sex[is.na(data$Sex)] <- names(which.max(table(data$Sex)))[1]
  }
  
  ########### Select columns to keep ###############
  data <- dplyr::select(data, person_id, HearingDate, CodeSection, codesection,
                        ChargeType, chargetype, Class, DispositionCode, disposition,
                        Plea, Race, Sex, fips, convictions:expungable, old_expungable, 
                        expungable_no_lifetimelimit, reason,
                        sameday, lifetime)
#  return(data)
#}
data

person_id,HearingDate,CodeSection,codesection,ChargeType,chargetype,Class,DispositionCode,disposition,Plea,⋯,within7,within10,class1_2,class3_4,expungable,old_expungable,expungable_no_lifetimelimit,reason,sameday,lifetime
<chr>,<date>,<chr>,<fct>,<chr>,<fct>,<chr>,<chr>,<fct>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<fct>,<lgl>,<fct>,<chr>,<lgl>,<lgl>
51220000000305,2010-05-03,46.2-613(2),covered elsewhere,Misdemeanor,Misdemeanor,1,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,Automatic,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date,FALSE,FALSE
51220000000305,2010-05-03,A.46.2-707,covered elsewhere,Misdemeanor,Misdemeanor,3,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Automatic,FALSE,Automatic,Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date,FALSE,FALSE
51220000000305,2010-05-03,46.2-300,covered elsewhere,Misdemeanor,Misdemeanor,2,Guilty,Conviction,Guilty,⋯,FALSE,FALSE,FALSE,FALSE,Not eligible,FALSE,Automatic,"Conviction of misdemeanor charges listed in 19.2-392.6 B with no convictions within 7 years from disposition date; HOWEVER, the outcome is changed to not eligible because the lifetime limit of two expungements has been exceeded",FALSE,TRUE


In [ ]:
# install pkgr
cd ~
wget https://github.com/metrumresearchgroup/pkgr/releases/download/v2.0.1/pkgr_2.0.1_linux_amd64.tar.gz -O pkgr.tar.gz
tar xzf pkgr.tar.gz pkgr
chmod +x pkgr

# clone the repo
git clone https://github.com/code-for-charlottesville/LAJC-expungement.git

# go into the repo and install the packages
cd LAJC-expungement
~/pkgr install 

In [47]:
sample_n(all_ids, 10)

person_id
<chr>
301171000000416
208070000000932
39020000000298
252110000000302
143051000000314
137220000000608
145171000000354
353071000000072
251030000000097


In [ ]:
INFO[0002] Successfully Installed.                       package=bit64 remaining=56 repo=MPN version=4.0.5
ERRO[0002] cmd output                                    exitCode=1 package=curl stderr="* installing *source* package ‘curl’ ...\n** package ‘curl’ successfully unpacked and MD5 sums checked\n** using staged installation\nERROR: configuration failed for package ‘curl’\n* removing ‘/tmp/OMSHHKECMQWN/curl’\n" stdout="Found pkg-config cflags and libs!\nUsing PKG_CFLAGS=-I/usr/include/x86_64-linux-gnu\nUsing PKG_LIBS=-lcurl\n------------------------- ANTICONF ERROR ---------------------------\nConfiguration failed because libcurl was not found. Try installing:\n * deb: libcurl4-openssl-dev (Debian, Ubuntu, etc)\n * rpm: libcurl-devel (Fedora, CentOS, RHEL)\n * csw: libcurl_dev (Solaris)\nIf libcurl is already installed, check that 'pkg-config' is in your\nPATH and PKG_CONFIG_PATH contains a libcurl.pc file. If pkg-config\nis unavailable you can set INCLUDE_DIR and LIB_DIR manually via:\nR CMD INSTALL --configure-vars='INCLUDE_DIR=... LIB_DIR=...'\n--------------------------------------------------------------------\n"
WARN[0002] error installing                              err="exit status 1"